In [1]:
# from BCBio.GFF import GFFExaminer

https://gffpandas.readthedocs.io/en/latest/tutorial.html

In [1]:
import gffpandas.gffpandas as gffpd
import pandas as pd

In [2]:
annotation = gffpd.read_gff3('YPS1009_S288C_v4_polished.gff')
annot_s288c = gffpd.read_gff3('saccharomyces_cerevisiae_R64-3.gff')

C:\Users\user\anaconda3\lib\site-packages\gffpandas\gffpandas.py:32: DtypeWarning: Columns (1,2,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_table(self._gff_file, comment='#',


In [3]:
annotation.df['type'].value_counts()

CDS                                7178
gene                               6707
mRNA                               6707
noncoding_exon                      493
tRNA_gene                           321
tRNA                                321
transposable_element_gene            99
transposable_element                 99
snoRNA                               77
snoRNA_gene                          77
plus_1_translational_frameshift      47
five_prime_UTR_intron                24
ncRNA_gene                           23
ncRNA                                23
snRNA_gene                            6
snRNA                                 6
uORF                                  6
telomeric_repeat                      3
intein_encoding_region                1
Name: type, dtype: int64

In [4]:
annot_s288c.df['type'].value_counts()

mRNA                                  11119
CDS                                    7066
gene                                   6607
ARS                                     543
noncoding_exon                          489
long_terminal_repeat                    384
intron                                  378
tRNA                                    299
tRNA_gene                               299
ARS_consensus_sequence                  196
transposable_element_gene                91
transposable_element                     91
region                                   78
snoRNA_gene                              77
snoRNA                                   77
telomere                                 64
LTR_retrotransposon                      50
plus_1_translational_frameshift          47
X_element                                32
centromere                               32
telomeric_repeat                         31
X_element_combinatorial_repeat           28
rRNA                            

## Make df by features YPS1009

In [5]:
TE_df = annotation.filter_feature_of_type(['transposable_element_gene'])
sno_df = annotation.filter_feature_of_type(['snoRNA_gene'])
tRNA_df = annotation.filter_feature_of_type(['tRNA_gene'])
gene_df = annotation.filter_feature_of_type(['gene'])

In [6]:
gene_attr = gene_df.attributes_to_columns()
gene = gene_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'gene', 'low_identity', 'orf_classification',
       'partial_mapping', 'sequence_ID', 'valid_ORFs']]

In [7]:
gene_attr['valid_ORFs'].value_counts()
# so valid_ORFs in gene does count itself and I should look carefully at the 439 genes with no valid ORF

1    6175
0     530
Name: valid_ORFs, dtype: int64

In [8]:
TE_attr = TE_df.attributes_to_columns()
sno_attr = sno_df.attributes_to_columns()
tRNA_attr = tRNA_df.attributes_to_columns()

In [9]:
tRNA_attr.columns

Index(['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'Alias', 'ID', 'Name', 'Note', 'Ontology_term',
       'copy_num_ID', 'coverage', 'curie', 'dbxref', 'display',
       'extra_copy_number', 'gene', 'sequence_ID'],
      dtype='object')

In [10]:
TE = TE_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'sequence_ID', 'valid_ORFs']]

sno = sno_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'gene', 'sequence_ID']]

tRNA = tRNA_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'gene', 'sequence_ID']]

In [11]:
# This code just convert the chromosome numbering to integer and remove mitochondrial genes.
def lat_to_arabic(DF):
    Latin_to_arabic = {'I': 1, 'II':2, 'III':3, 'IV':4, 'V':5, 'VI':6, 'VII':7, 'VIII':8, 'IX':9, 'X':10,
                      'XI':11, 'XII':12, 'XIII':13, 'XIV':14, 'XV':15, 'XVI':16, 'Mito': 'mitoch', 'MANY': 7}
    seq_id = DF['seq_id']
    ch = []
    contig = []
    for i in seq_id:
        chrom, cont = i.split('_')
        contig.append(cont)
        chrom = chrom.replace('chr', '')
        if '-' in chrom:
            chrom, e = chrom.split('-')
        ch.append(Latin_to_arabic[chrom])
    DF['Chromosome'] = ch
    DF['Contig'] = contig
    return DF

In [12]:
gene = lat_to_arabic(gene)

C:\Users\user\AppData\Local\Temp\ipykernel_12224\3764198105.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Chromosome'] = ch
C:\Users\user\AppData\Local\Temp\ipykernel_12224\3764198105.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Contig'] = contig


In [13]:
TE = lat_to_arabic(TE)
sno = lat_to_arabic(sno)
tRNA = lat_to_arabic(tRNA)

C:\Users\user\AppData\Local\Temp\ipykernel_12224\3764198105.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Chromosome'] = ch
C:\Users\user\AppData\Local\Temp\ipykernel_12224\3764198105.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Contig'] = contig
C:\Users\user\AppData\Local\Temp\ipykernel_12224\3764198105.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [14]:
gene.head()

,seq_id,start,end,Name,Alias,Note,Ontology_term,score,strand,phase,...,display,extra_copy_number,gene,low_identity,orf_classification,partial_mapping,sequence_ID,valid_ORFs,Chromosome,Contig
0,chrII-2_tig00004927,8297,10119,YBR298C,"MAL31,MAL3T,MALT,maltose%20permease",Maltose%20permease%3B%20high-affinity%20maltos...,"GO:0000017,GO:0005352,GO:0071944,SO:0000704",.,-,.,...,Maltose%20permease,1,MAL31,None,Verified,None,0.946,0,2,tig00004927
3,chrII-2_tig00004927,10675,10894,YBR298C-A,None,Putative%20protein%20of%20unknown%20function%3...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,-,.,...,Putative%20protein%20of%20unknown%20function,1,None,None,Uncharacterized,None,0.981,0,2,tig00004927
6,chrII-2_tig00004927,10985,12717,YBR299W,"MAL32,MAL3S,MALS,alpha-glucosidase%20MAL32",Maltase%20%28alpha-D-glucosidase%29%3B%20induc...,"GO:0000025,GO:0004575,GO:0005886,GO:0005987,GO...",.,+,.,...,Maltase%20%28alpha-D-glucosidase%29,1,MAL32,None,Verified,None,0.981,0,2,tig00004927
9,chrII-2_tig00004927,14242,14700,YGR293C,None,Dubious%20open%20reading%20frame%3B%20unlikely...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,-,.,...,Dubious%20open%20reading%20frame,2,None,None,Dubious,None,0.989,0,2,tig00004927
12,chrII-2_tig00004927,14660,15022,YGR294W,"PAU12,seripauperin%20PAU12",Protein%20of%20unknown%20function%3B%20member%...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,+,.,...,Protein%20of%20unknown%20function,1,PAU12,None,Verified,None,0.983,1,2,tig00004927


In [15]:
def clean20(DF):
    note = DF['Note']
    dis = DF['display']
    clean = []
    display = []
    for n in note:
        clean.append(n.replace('%20', ' '))
    for d in dis:
        display.append(str(d).replace('%20', ' '))
    DF['Note'] = clean
    DF['display'] = display
    return DF

In [16]:
gene = clean20(gene)
TE = clean20(TE)
sno = clean20(sno)
tRNA = clean20(tRNA)

C:\Users\user\AppData\Local\Temp\ipykernel_12224\741321415.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Note'] = clean
C:\Users\user\AppData\Local\Temp\ipykernel_12224\741321415.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['display'] = display


## Df by attribute for S288C

In [17]:
TE_df_s288c = annot_s288c.filter_feature_of_type(['transposable_element_gene'])
sno_df_s288c = annot_s288c.filter_feature_of_type(['snoRNA_gene'])
tRNA_df_s288c = annot_s288c.filter_feature_of_type(['tRNA_gene'])
gene_df_s288c = annot_s288c.filter_feature_of_type(['gene'])

In [18]:
gene_attr_s288c = gene_df_s288c.attributes_to_columns()
TE_attr_s288c = TE_df_s288c.attributes_to_columns()
sno_attr_s288c = sno_df_s288c.attributes_to_columns()
tRNA_attr_s288c = tRNA_df_s288c.attributes_to_columns()

In [19]:
TE_attr_s288c.columns

Index(['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'Alias', 'ID', 'Name', 'Note', 'Ontology_term', 'curie',
       'dbxref', 'display'],
      dtype='object')

In [20]:
gene_s288c = gene_attr_s288c[['seq_id', 'start', 'end', 'Name', 'gene',  'Alias',  'Note', 'Ontology_term', 'strand', 'phase', 'curie',
        'orf_classification', 'display']]

TE_s288c = TE_attr_s288c[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
      'curie', 'display']]

sno_s288c = sno_attr_s288c[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
      'curie', 'display']]

tRNA_s288c = tRNA_attr_s288c[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
      'curie', 'display']]

In [21]:
def lat_to_arabic_S288(df):
    # This code just convert the chromosome numbering to integer and remove mitochondrial genes.
    Latin_to_arabic = {'I': 1, 'II':2, 'III':3, 'IV':4, 'V':5, 'VI':6, 'VII':7, 'VIII':8, 'IX':9, 'X':10,
                      'XI':11, 'XII':12, 'XIII':13, 'XIV':14, 'XV':15, 'XVI':16, 'Mito': 'mitoch'}
    chLat = df['seq_id']
    ch = []
    for chrom in chLat:
        chrom = chrom.replace('chr', '')
        ch.append(Latin_to_arabic[chrom])
    df['Chromosome'] = ch
    return df

In [22]:
gene_s288c = lat_to_arabic_S288(gene_s288c)
TE_s288c = lat_to_arabic_S288(TE_s288c)
sno_s288c = lat_to_arabic_S288(sno_s288c)
tRNA_s288c = lat_to_arabic_S288(tRNA_s288c)

C:\Users\user\AppData\Local\Temp\ipykernel_12224\760545625.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Chromosome'] = ch
C:\Users\user\AppData\Local\Temp\ipykernel_12224\760545625.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Chromosome'] = ch
C:\Users\user\AppData\Local\Temp\ipykernel_12224\760545625.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [23]:
gene_s288c = clean20(gene_s288c)
TE_s288c = clean20(TE_s288c)
sno_s288c = clean20(sno_s288c)
tRNA_s288c = clean20(tRNA_s288c)

C:\Users\user\AppData\Local\Temp\ipykernel_12224\741321415.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Note'] = clean
C:\Users\user\AppData\Local\Temp\ipykernel_12224\741321415.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['display'] = display


# TE

In [24]:
TE.to_excel('TE_liftoff4_yps1009.xlsx')

In [25]:
TE_s288c['Chromosome'].value_counts()

4     17
7     10
16     9
12     8
13     8
15     8
2      6
10     6
5      4
8      4
14     4
1      2
3      2
6      2
9      1
Name: Chromosome, dtype: int64

In [26]:
TE['Chromosome'].value_counts()

7     54
12    14
4     12
5      8
2      2
6      2
15     2
3      1
1      1
8      1
16     1
10     1
Name: Chromosome, dtype: int64

In [27]:
TE['valid_ORFs'].value_counts()

1    61
0    38
Name: valid_ORFs, dtype: int64

In [28]:
TE_count = TE['Chromosome'].value_counts().to_frame()
TE_count = TE_count.rename(columns = {'Chromosome': 'yps1009 TE'})
TE_count_s = TE_s288c['Chromosome'].value_counts().to_frame()
TE_count_s = TE_count_s.rename(columns = {'Chromosome': 'S288C TE'})

In [29]:
TE_count = TE_count_s.join(TE_count)

In [30]:
TE_count

,S288C TE,yps1009 TE
4,17,12.0
7,10,54.0
16,9,1.0
12,8,14.0
13,8,NaN
15,8,2.0
2,6,2.0
10,6,1.0
5,4,8.0
8,4,1.0


In [31]:
TE7_yps = list(TE[TE['Chromosome'] == 7]['Name'])
TE7_s288 = list(TE_s288c[TE_s288c['Chromosome'] == 7]['Name'])

In [32]:
set(TE7_s288) & set(TE7_yps)

{'YGR027W-A', 'YGR027W-B', 'YGR038C-A', 'YGR038C-B', 'YGR161C-C'}

In [33]:
TE['valid_ORFs'].value_counts()

1    61
0    38
Name: valid_ORFs, dtype: int64

In [34]:
TE[TE['Chromosome'] == 1]

,seq_id,start,end,Name,Alias,Note,Ontology_term,score,strand,phase,copy_num_ID,coverage,curie,display,extra_copy_number,sequence_ID,valid_ORFs,Chromosome,Contig
5939,chrI_tig00000033,177837,179152,YDR034C-C,gag%20protein,Retrotransposon TYA Gag gene co-transcribed wi...,"GO:0000943,GO:0003723,GO:0032197,SO:0000704",.,+,.,YDR034C-C_4,0.999,SGD:S000007344,Retrotransposon TYA Gag gene co-transcribed wi...,4,0.973,0,1,tig00000033


# snoRNA

In [35]:
sno_count = sno['Chromosome'].value_counts().to_frame()
sno_count = sno_count.rename(columns = {'Chromosome': 'yps1009 snoRNA'})
sno_count_s = sno_s288c['Chromosome'].value_counts().to_frame()
sno_count_s = sno_count_s.rename(columns = {'Chromosome': 'S288C snoRNA'})
sno_count = sno_count_s.join(sno_count)

In [36]:
sno_count

,S288C snoRNA,yps1009 snoRNA
13,13,13
15,11,11
12,7,7
7,6,6
16,6,6
5,5,5
10,5,5
11,5,5
14,5,5
3,4,4


# tRNA

In [37]:
tRNA['extra_copy_number'].value_counts()

0    299
1     22
Name: extra_copy_number, dtype: int64

In [38]:
tRNA_count = tRNA['Chromosome'].value_counts().to_frame()
tRNA_count = tRNA_count.rename(columns = {'Chromosome': 'yps1009 tRNARNA'})
tRNA_count_s = tRNA_s288c['Chromosome'].value_counts().to_frame()
tRNA_count_s = tRNA_count_s.rename(columns = {'Chromosome': 'S288C tRNARNA'})
tRNA_count = tRNA_count_s.join(tRNA_count)

In [39]:
tRNA_count

,S288C tRNARNA,yps1009 tRNARNA
7,36,38
4,28,30
mitoch,24,26
10,24,27
12,21,22
13,21,22
5,20,21
15,20,20
16,17,18
11,16,18


# GENE

In [40]:
gene['orf_classification'].value_counts()

Verified           5249
Uncharacterized     737
Dubious             719
Name: orf_classification, dtype: int64

In [41]:
dubious = gene[gene['orf_classification'] == 'Dubious']
dubious['valid_ORFs'].value_counts()

1    524
0    195
Name: valid_ORFs, dtype: int64

In [42]:
uncharacterized = gene[gene['orf_classification'] == 'Uncharacterized']
uncharacterized['valid_ORFs'].value_counts()

1    564
0    173
Name: valid_ORFs, dtype: int64

# Verified ORF

In [43]:
verified = gene[gene['orf_classification'] == 'Verified']

In [44]:
verified['valid_ORFs'].value_counts()

1    5087
0     162
Name: valid_ORFs, dtype: int64

In [77]:
verified_s288c = gene_s288c[gene_s288c['orf_classification'] == 'Verified']

In [79]:
verified_s288c.shape

(5216, 14)

In [80]:
verified_valid = verified[verified['valid_ORFs'] == '1']
verified_unvalid = verified[verified['valid_ORFs'] == '0']

verified_valid_count = verified_valid['Chromosome'].value_counts().to_frame()
verified_valid_count = verified_valid_count.rename(columns={'Chromosome': 'Number valid genes'})
verified_unvalid_count = verified_unvalid['Chromosome'].value_counts().to_frame()
verified_unvalid_count = verified_unvalid_count.rename(columns={'Chromosome': 'Number unvalid genes'})

In [81]:
verified_unvalid.head(2)

,seq_id,start,end,Name,Alias,Note,Ontology_term,score,strand,phase,...,display,extra_copy_number,gene,low_identity,orf_classification,partial_mapping,sequence_ID,valid_ORFs,Chromosome,Contig
0,chrII-2_tig00004927,8297,10119,YBR298C,"MAL31,MAL3T,MALT,maltose%20permease",Maltose permease%3B high-affinity maltose tran...,"GO:0000017,GO:0005352,GO:0071944,SO:0000704",.,-,.,...,Maltose permease,1,MAL31,None,Verified,None,0.946,0,2,tig00004927
6,chrII-2_tig00004927,10985,12717,YBR299W,"MAL32,MAL3S,MALS,alpha-glucosidase%20MAL32",Maltase %28alpha-D-glucosidase%29%3B inducible...,"GO:0000025,GO:0004575,GO:0005886,GO:0005987,GO...",.,+,.,...,Maltase %28alpha-D-glucosidase%29,1,MAL32,None,Verified,None,0.981,0,2,tig00004927


In [82]:
unval_dic = {}
for ind, row in verified_unvalid.iterrows():
    if row['Chromosome'] in unval_dic:
        unval_dic[row['Chromosome']] = unval_dic[row['Chromosome']] + [row['Name']]
    else:
        unval_dic[row['Chromosome']] = [row['Name']]    

In [83]:
verified_sum = verified_valid_count.join(verified_unvalid_count)

In [84]:
tmp = []
for ind in verified_sum.index:
    tmp.append(unval_dic[ind])

In [85]:
verified_sum['unvalid genes'] = tmp

In [ ]:
tmp = []
count = []
for Ch in verified_sum.index:
    s288 = verifiedS288C[verifiedS288C['S288C Chromosome'] == Ch]
    yps = verified[verified['Chromosome'] == Ch]
    tmp.append(list(set(s288.index) - set(yps['Name'])))
    count.append(len(list(set(s288.index) - set(yps['Name']))))

In [ ]:
verified_sum['Number of missing genes'] = count
verified_sum['Missing verified genes'] = tmp

In [ ]:
verified_sum

In [ ]:
verified_sum.sum

In [ ]:
verified_sum.to_excel('verified_genes_summary.xlsx')

# Blast

In [ ]:

# Import Biopython tools for running local BLASTX
from Bio.Blast.Applications import NcbiblastnCommandline

# Chromosome IV
[YDR385W, YDR542W, YDL248W, YDR545W, YDL244W]

In [ ]:
cmd_blastn = NcbiblastnCommandline(query= 'fasta_file/S288C_YDR038C_ENA5_genomic.fsa', out='Blast_results/YDR038C.tab', evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')

In [ ]:
cmd_blastn = NcbiblastnCommandline(query= 'fasta_file/S288C_YDR039C_ENA2_genomic.fsa', out='Blast_results/YDR039C.tab', evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')
stdout, stderr = cmd_blastn()
# Check STDOUT, STDERR
print("STDOUT: %s" % stdout)
print("STDERR: %s" % stderr)

In [ ]:
cmd_blastn = NcbiblastnCommandline(query= 'fasta_file/S288C_YDR545W_YRF1-1_genomic.fsa', out='Blast_results/YDR545W.tab', evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')
stdout, stderr = cmd_blastn()
# Check STDOUT, STDERR
print("STDOUT: %s" % stdout)
print("STDERR: %s" % stderr)

In [ ]:
verified[verified['Name'] == 'YDR545W']

Chromosome 4: 2 near centromere, 1 at the very end of teleomere
at centromere: ENA2 and ENA5 : next to each other, Na(+)-exporting P-type ATPase ENA2
ENA5 align with 90% identity to contig tig00000001 IV
ENA2 align to exactly the same region of contig IV, with 90% identity, duplication in s288C?

YRF1 align to contig >tig00000015 on chromosome V with 97% identity, at the very end/beginning (complement?)
Liftoff did not detect YRF1 on chromosome V. 

Is it a valid ORF, is there easy tool to check that?

# Chromosome I

In [ ]:
cmd_blastn = NcbiblastnCommandline(query= 'fasta_file/S288C_YAL063C_FLO9_genomic.fsa', out='Blast_results/YAL063C.tab', evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')
stdout, stderr = cmd_blastn()
# Check STDOUT, STDERR
print("STDOUT: %s" % stdout)
print("STDERR: %s" % stderr)
cmd_blastn = NcbiblastnCommandline(query= 'fasta_file/S288C_YAR027W_UIP3_genomic.fsa', out='Blast_results/YAR027W.tab', evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')
stdout, stderr = cmd_blastn()
# Check STDOUT, STDERR
print("STDOUT: %s" % stdout)
print("STDERR: %s" % stderr)

FLO9:
2 matches: score    e-val
tig00000033 I 2691    0.0  12 gap   86% identity
tig00000017 VIII  1718    0.0  57 gap 

UIP3: 
2 matches
tig00000033 I  233     3e-61  87% identity 6 gap alignment without stop codon at the end.
tig00004928 VII  171     3e-42  82% 6 gap, alignment finish with stop codon

# Chromosome II

In [ ]:
cmd_blastn = NcbiblastnCommandline(query= 'fasta_file/S288C_YBL108C-A_PAU9_genomic.fsa', out='Blast_results/YBL108C.tab', evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')
stdout, stderr = cmd_blastn()
# Check STDOUT, STDERR
print("STDOUT: %s" % stdout)
print("STDERR: %s" % stderr)
cmd_blastn = NcbiblastnCommandline(query= 'fasta_file/S288C_YBL111C_YBL111C_genomic.fsa', out='Blast_results/YBL111C.tab', evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')
stdout, stderr = cmd_blastn()
# Check STDOUT, STDERR
print("STDOUT: %s" % stdout)
print("STDERR: %s" % stderr)
cmd_blastn = NcbiblastnCommandline(query= 'fasta_file/S288C_YBR298C_MAL31_genomic.fsa', out='Blast_results/YBR298C.tab', evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')
stdout, stderr = cmd_blastn()
# Check STDOUT, STDERR
print("STDOUT: %s" % stdout)
print("STDERR: %s" % stderr)

In [ ]:
verified[verified['Name'] == 'YBL111C']

# Duplicated genes???



In [ ]:
# Many duplicated genes in this version
gene_attr['extra_copy_number'].value_counts()

In [ ]:
gene_attr[gene_attr['extra_copy_number'] > '1']

In [ ]:
gene_attr[gene_attr.duplicated(subset=['Name'], keep=False) == True]

Duplicated genes:

ATP8/Q0080: Subunit 8 of the F0 sector of mitochondrial F1F0 ATP synthase; encoded on the mitochondrial genome; ATP8 and ATP6 mRNAs are not translated in the absence of the F1 sector of ATPase

OLI1/Q0130: F0-ATP synthase subunit c (ATPase-associated proteolipid); encoded on the mitochondrial genome; mutation confers oligomycin resistance; expression is specifically dependent on the nuclear genes AEP1 and AEP2 1 2 3 4
Name Description
OLIgomycin resistance 1

Both on mitochondria so not that relevant to my project.

PAU9/YBL108C-A: Protein of unknown function; member of the seripauperin multigene family encoded mainly in subtelomeric regions; SWAT-GFP and mCherry fusion proteins localize to the endoplasmic reticulum and vacuole respectively

YHR213W-A: Putative protein of unknown function; identified by gene-trapping, microarray-based expression analysis, and genome-wide homology searching 